# Complete Robust Anomaly Detection System
## End-to-End Workflow with Statistical Rigor

This notebook provides a **complete, production-ready workflow** for anomaly detection with:
- Multiple algorithms (Gaussian, Isolation Forest, One-Class SVM, LOF, Elliptic Envelope)
- Advanced imbalance handling (SMOTE, ADASYN, etc.)
- Comprehensive statistical analysis
- Model comparison and evaluation
- Professional visualizations

---

## Table of Contents
1. [Setup & Installation](#setup)
2. [Load and Explore Data](#data)
3. [Statistical Analysis](#stats)
4. [Run Complete Pipeline](#pipeline)
5. [Model Comparison](#comparison)
6. [Detailed Evaluation](#evaluation)
7. [Feature Analysis](#features)
8. [Production Deployment](#production)
9. [Summary & Next Steps](#summary)

---
## 1. Setup & Installation <a id='setup'></a>

First, let's install all required packages and import libraries.

In [ ]:
# Install required packages (run once)
!pip install -q numpy pandas scipy scikit-learn imbalanced-learn xgboost matplotlib seaborn

In [ ]:
# Import all required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Configure visualization style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 11
%matplotlib inline

print("✅ All imports successful!")
print(f"📊 NumPy version: {np.__version__}")
print(f"📊 Pandas version: {pd.__version__}")

In [ ]:
# Import the robust anomaly detection system
from robust_anomaly_detection import (
    run_robust_anomaly_detection,
    RobustAnomalyDetector,
    StatisticalAnalyzer,
    ComprehensiveEvaluator,
    OutlierDetector
)

print("✅ Robust Anomaly Detection System loaded successfully!")

---
## 2. Load and Explore Data <a id='data'></a>

We'll create a synthetic dataset that mimics real-world anomaly detection scenarios with class imbalance.

In [ ]:
from sklearn.datasets import make_classification

# Generate synthetic anomaly detection dataset
print("🔧 Generating synthetic dataset...")
print("="*80)

# Dataset parameters
N_SAMPLES = 2000
N_FEATURES = 25
ANOMALY_RATIO = 0.12  # 12% anomalies (imbalanced)

X, y = make_classification(
    n_samples=N_SAMPLES,
    n_features=N_FEATURES,
    n_informative=18,
    n_redundant=4,
    n_repeated=0,
    n_classes=2,
    n_clusters_per_class=3,
    weights=[1-ANOMALY_RATIO, ANOMALY_RATIO],
    flip_y=0.03,
    class_sep=0.8,
    random_state=RANDOM_STATE
)

# Create feature names (domain-specific for context)
feature_names = [
    'temperature', 'humidity', 'wind_speed', 'pressure',
    'visibility', 'precipitation', 'cloud_cover', 'uv_index',
    'air_quality_pm25', 'air_quality_pm10', 'air_quality_o3',
    'noise_level', 'traffic_density', 'pedestrian_count',
    'vegetation_index', 'soil_moisture', 'solar_radiation',
    'gas_sensor_1', 'gas_sensor_2', 'gas_sensor_3',
    'thermal_sensor_1', 'thermal_sensor_2', 'motion_sensor',
    'vibration_sensor', 'light_sensor'
]

# Create DataFrame
X_df = pd.DataFrame(X, columns=feature_names)
y_series = pd.Series(y, name='target')

print(f"✅ Dataset created successfully!")
print(f"   Total samples: {len(X_df):,}")
print(f"   Number of features: {X_df.shape[1]}")
print(f"   Normal samples: {np.sum(y == 0):,} ({np.sum(y == 0)/len(y)*100:.1f}%)")
print(f"   Anomaly samples: {np.sum(y == 1):,} ({np.sum(y == 1)/len(y)*100:.1f}%)")
print(f"   Imbalance ratio: {np.sum(y == 0)/np.sum(y == 1):.2f}:1")

In [ ]:
# Display first few rows
display_df = X_df.copy()
display_df['target'] = y

print("\n📋 Sample Data (first 10 rows):")
display_df.head(10)

In [ ]:
# Statistical summary
print("\n📊 Statistical Summary:")
X_df.describe().T

In [ ]:
# Visualize class distribution
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Count plot
unique, counts = np.unique(y, return_counts=True)
colors = ['#2ecc71', '#e74c3c']
axes[0].bar(['Normal', 'Anomaly'], counts, color=colors, edgecolor='black', linewidth=1.5)
axes[0].set_ylabel('Count', fontsize=13, fontweight='bold')
axes[0].set_title('Class Distribution (Counts)', fontsize=14, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)
for i, count in enumerate(counts):
    axes[0].text(i, count + 20, f'{count:,}', ha='center', fontsize=12, fontweight='bold')

# Percentage plot
percentages = counts / len(y) * 100
axes[1].pie(percentages, labels=['Normal', 'Anomaly'], autopct='%1.1f%%',
           colors=colors, startangle=90, textprops={'fontsize': 12, 'fontweight': 'bold'})
axes[1].set_title('Class Distribution (Percentage)', fontsize=14, fontweight='bold')

# Imbalance ratio visualization
ratio = counts[0] / counts[1]
axes[2].barh(['Imbalance\nRatio'], [ratio], color='#3498db', edgecolor='black', linewidth=1.5)
axes[2].set_xlabel('Ratio (Normal:Anomaly)', fontsize=13, fontweight='bold')
axes[2].set_title('Class Imbalance', fontsize=14, fontweight='bold')
axes[2].text(ratio/2, 0, f'{ratio:.2f}:1', ha='center', va='center', 
            fontsize=14, fontweight='bold', color='white')
axes[2].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n⚠️  WARNING: Significant class imbalance detected ({ratio:.2f}:1)")
print(f"   Recommendation: Use SMOTE, ADASYN, or other imbalance handling techniques")

In [ ]:
# Visualize feature distributions
print("\n📊 Feature Distribution Analysis")
print("="*80)

# Select 6 random features to visualize
sample_features = np.random.choice(feature_names, 6, replace=False)

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()

for idx, feature in enumerate(sample_features):
    # Separate by class
    normal_data = X_df[y == 0][feature]
    anomaly_data = X_df[y == 1][feature]
    
    # Plot histograms
    axes[idx].hist(normal_data, bins=30, alpha=0.6, label='Normal', color='#2ecc71', edgecolor='black')
    axes[idx].hist(anomaly_data, bins=30, alpha=0.6, label='Anomaly', color='#e74c3c', edgecolor='black')
    axes[idx].set_xlabel(feature, fontsize=11)
    axes[idx].set_ylabel('Frequency', fontsize=11)
    axes[idx].set_title(f'Distribution: {feature}', fontsize=12, fontweight='bold')
    axes[idx].legend()
    axes[idx].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Check for missing values and data quality
print("\n🔍 Data Quality Check")
print("="*80)

missing_counts = X_df.isnull().sum()
print(f"Missing values: {missing_counts.sum()}")

if missing_counts.sum() > 0:
    print("\nFeatures with missing values:")
    print(missing_counts[missing_counts > 0])
else:
    print("✅ No missing values detected")

# Check for duplicates
duplicates = X_df.duplicated().sum()
print(f"\nDuplicate rows: {duplicates}")

if duplicates > 0:
    print(f"   Recommendation: Consider removing {duplicates} duplicate rows")
else:
    print("✅ No duplicate rows detected")

print("\n✅ Data quality check complete!")

---
## 3. Statistical Analysis <a id='stats'></a>

Perform comprehensive statistical analysis including:
- Distribution testing (normality)
- Feature significance testing
- Effect size analysis

In [ ]:
from sklearn.model_selection import train_test_split

# Split data for analysis
X_train, X_test, y_train, y_test = train_test_split(
    X_df, y, test_size=0.25, random_state=RANDOM_STATE, stratify=y
)

print(f"\n📊 Data Split:")
print(f"   Training set: {len(X_train):,} samples")
print(f"   Test set: {len(X_test):,} samples")

# Initialize statistical analyzer
analyzer = StatisticalAnalyzer(alpha=0.05)

# Perform comprehensive EDA
dist_df, sig_df = analyzer.comprehensive_eda(
    X_train.values, 
    y_train, 
    feature_names
)

In [ ]:
# Display distribution test results
print("\n📊 Distribution Test Results (Top 10 Non-Normal Features)")
print("="*80)

non_normal = dist_df[~dist_df['is_normal_shapiro']].nsmallest(10, 'shapiro_p')
print(non_normal[['feature', 'shapiro_p', 'skewness', 'kurtosis']].to_string(index=False))

print(f"\n✅ Normal features: {dist_df['is_normal_shapiro'].sum()}/{len(dist_df)}")
print(f"⚠️  Non-normal features: {(~dist_df['is_normal_shapiro']).sum()}/{len(dist_df)}")
print("\n💡 Recommendation: Use RobustScaler or PowerTransformer for preprocessing")

In [ ]:
# Display feature significance results
print("\n📊 Feature Significance Analysis (Top 15 Most Significant)")
print("="*80)

top_significant = sig_df.nsmallest(15, 'mannwhitney_p')[
    ['feature', 'cohens_d', 'effect_size', 'mannwhitney_p', 'is_significant']
]
print(top_significant.to_string(index=False))

# Count significant features
n_significant = sig_df['is_significant'].sum()
print(f"\n✅ Statistically significant features: {n_significant}/{len(sig_df)} (p < 0.05)")

In [ ]:
# Visualize feature significance
fig, axes = plt.subplots(1, 2, figsize=(18, 7))

# Top 15 by p-value
top_15 = sig_df.nsmallest(15, 'mannwhitney_p')

# P-values (log scale)
axes[0].barh(range(len(top_15)), -np.log10(top_15['mannwhitney_p']), 
            color='#3498db', edgecolor='black', linewidth=1)
axes[0].set_yticks(range(len(top_15)))
axes[0].set_yticklabels(top_15['feature'])
axes[0].set_xlabel('-log10(p-value)', fontsize=13, fontweight='bold')
axes[0].set_title('Feature Significance (p-values)', fontsize=14, fontweight='bold')
axes[0].axvline(-np.log10(0.05), color='red', linestyle='--', linewidth=2, label='α=0.05')
axes[0].legend(fontsize=11)
axes[0].invert_yaxis()
axes[0].grid(axis='x', alpha=0.3)

# Effect sizes (Cohen's d)
colors_effect = ['#e74c3c' if abs(d) >= 0.8 else '#f39c12' if abs(d) >= 0.5 else '#95a5a6' 
                for d in top_15['cohens_d']]
axes[1].barh(range(len(top_15)), np.abs(top_15['cohens_d']), 
            color=colors_effect, edgecolor='black', linewidth=1)
axes[1].set_yticks(range(len(top_15)))
axes[1].set_yticklabels(top_15['feature'])
axes[1].set_xlabel("|Cohen's d| (Effect Size)", fontsize=13, fontweight='bold')
axes[1].set_title('Feature Effect Sizes', fontsize=14, fontweight='bold')
axes[1].axvline(0.8, color='#e74c3c', linestyle='--', linewidth=2, label='Large (0.8)')
axes[1].axvline(0.5, color='#f39c12', linestyle='--', linewidth=2, label='Medium (0.5)')
axes[1].axvline(0.2, color='#95a5a6', linestyle='--', linewidth=2, label='Small (0.2)')
axes[1].legend(fontsize=11)
axes[1].invert_yaxis()
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Interpretation:")
print("   - Higher -log10(p-value) = More statistically significant")
print("   - |Cohen's d| ≥ 0.8 = Large effect (red)")
print("   - |Cohen's d| ≥ 0.5 = Medium effect (orange)")
print("   - |Cohen's d| ≥ 0.2 = Small effect (gray)")

---
## 4. Run Complete Pipeline <a id='pipeline'></a>

Now we'll run the complete robust anomaly detection pipeline which:
1. Preprocesses data with optimal scaling
2. Handles class imbalance
3. Trains multiple anomaly detection models
4. Evaluates and compares all models
5. Selects the best performing model

In [ ]:
# Run the complete robust anomaly detection pipeline
print("\n" + "="*100)
print("🚀 RUNNING COMPLETE ANOMALY DETECTION PIPELINE")
print("="*100)

detector, results = run_robust_anomaly_detection(
    X=X_df,
    y=y,
    test_size=0.25,
    scaling_method='robust',      # Use RobustScaler (resistant to outliers)
    imbalance_method='smote',     # Use SMOTE for class balancing
    models='all',                 # Train all 5 available models
    random_state=RANDOM_STATE
)

---
## 5. Model Comparison <a id='comparison'></a>

Compare all trained models across multiple metrics.

In [ ]:
# Display comprehensive model comparison
comparison_df = results['comparison']

print("\n" + "="*100)
print("📊 COMPREHENSIVE MODEL COMPARISON")
print("="*100)

# Select metrics to display
display_metrics = ['model', 'accuracy', 'precision', 'recall', 'f1', 
                  'balanced_accuracy', 'roc_auc', 'pr_auc', 'matthews_corrcoef']
display_metrics = [m for m in display_metrics if m in comparison_df.columns]

print(comparison_df[display_metrics].to_string(index=False))

print(f"\n🏆 BEST MODEL: {results['best_model'].upper()}")
best_row = comparison_df[comparison_df['model'] == results['best_model']].iloc[0]
print(f"   F1-Score: {best_row['f1']:.4f}")
print(f"   Precision: {best_row['precision']:.4f}")
print(f"   Recall: {best_row['recall']:.4f}")
print(f"   ROC-AUC: {best_row.get('roc_auc', 'N/A')}")

In [ ]:
# Visualize model comparison across multiple metrics
metrics_to_plot = ['precision', 'recall', 'f1', 'balanced_accuracy']
models = comparison_df['model'].values

fig, axes = plt.subplots(2, 2, figsize=(18, 12))
axes = axes.ravel()

for i, metric in enumerate(metrics_to_plot):
    values = comparison_df[metric].values
    
    # Color best model differently
    colors = ['#e74c3c' if model == results['best_model'] else '#3498db' 
             for model in models]
    
    bars = axes[i].barh(models, values, color=colors, edgecolor='black', linewidth=1.5)
    axes[i].set_xlabel(metric.replace('_', ' ').title(), fontsize=13, fontweight='bold')
    axes[i].set_title(f'{metric.replace("_", " ").title()} Comparison', 
                     fontsize=14, fontweight='bold')
    axes[i].set_xlim([0, 1])
    axes[i].grid(axis='x', alpha=0.3)
    
    # Add value labels
    for j, (v, bar) in enumerate(zip(values, bars)):
        axes[i].text(v + 0.02, j, f'{v:.3f}', va='center', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n📌 Note: Red bar indicates the best performing model")

In [ ]:
# Create radar chart for model comparison
from math import pi

# Select top 3 models
top_3_models = comparison_df.nlargest(3, 'f1')['model'].values
metrics_radar = ['precision', 'recall', 'f1', 'balanced_accuracy']

# Number of variables
num_vars = len(metrics_radar)
angles = [n / float(num_vars) * 2 * pi for n in range(num_vars)]
angles += angles[:1]

# Initialize plot
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))

colors_radar = ['#e74c3c', '#3498db', '#2ecc71']

for idx, model in enumerate(top_3_models):
    values = comparison_df[comparison_df['model'] == model][metrics_radar].values.flatten().tolist()
    values += values[:1]
    
    ax.plot(angles, values, 'o-', linewidth=2, label=model, color=colors_radar[idx])
    ax.fill(angles, values, alpha=0.15, color=colors_radar[idx])

# Fix axis labels
ax.set_xticks(angles[:-1])
ax.set_xticklabels([m.replace('_', ' ').title() for m in metrics_radar], fontsize=12)
ax.set_ylim(0, 1)
ax.set_title('Model Performance Comparison (Top 3)', fontsize=16, fontweight='bold', pad=20)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1), fontsize=11)
ax.grid(True)

plt.tight_layout()
plt.show()

---
## 6. Detailed Evaluation <a id='evaluation'></a>

Detailed evaluation of the best performing model.

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Get best model predictions
best_model = results['best_model']
y_pred_best = results['y_pred_best']
y_prob_best = results['y_prob_best']
y_test_best = results['y_test']

# Print classification report
print("\n" + "="*100)
print(f"📊 DETAILED CLASSIFICATION REPORT - {best_model.upper()}")
print("="*100)
print(classification_report(y_test_best, y_pred_best, 
                           target_names=['Normal', 'Anomaly'],
                           digits=4))

In [ ]:
# Confusion matrix visualization
cm = confusion_matrix(y_test_best, y_pred_best)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Count-based confusion matrix
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0],
           xticklabels=['Normal', 'Anomaly'],
           yticklabels=['Normal', 'Anomaly'],
           cbar_kws={'label': 'Count'},
           annot_kws={'fontsize': 14, 'fontweight': 'bold'})
axes[0].set_ylabel('True Label', fontsize=13, fontweight='bold')
axes[0].set_xlabel('Predicted Label', fontsize=13, fontweight='bold')
axes[0].set_title(f'Confusion Matrix (Counts) - {best_model}', fontsize=14, fontweight='bold')

# Percentage-based confusion matrix
cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
sns.heatmap(cm_percent, annot=True, fmt='.1f', cmap='Oranges', ax=axes[1],
           xticklabels=['Normal', 'Anomaly'],
           yticklabels=['Normal', 'Anomaly'],
           cbar_kws={'label': 'Percentage (%)'},
           annot_kws={'fontsize': 14, 'fontweight': 'bold'})
axes[1].set_ylabel('True Label', fontsize=13, fontweight='bold')
axes[1].set_xlabel('Predicted Label', fontsize=13, fontweight='bold')
axes[1].set_title(f'Confusion Matrix (%) - {best_model}', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

# Print confusion matrix breakdown
tn, fp, fn, tp = cm.ravel()
print("\n📊 Confusion Matrix Breakdown:")
print(f"   True Negatives (TN):  {tn:,} - Correctly identified normal samples")
print(f"   False Positives (FP): {fp:,} - Normal samples incorrectly flagged as anomalies")
print(f"   False Negatives (FN): {fn:,} - Anomalies missed by the model")
print(f"   True Positives (TP):  {tp:,} - Correctly identified anomalies")
print(f"\n   Specificity (True Negative Rate): {tn/(tn+fp):.4f}")
print(f"   Sensitivity (True Positive Rate): {tp/(tp+fn):.4f}")

In [ ]:
# ROC and Precision-Recall Curves
if y_prob_best is not None:
    from sklearn.metrics import roc_curve, precision_recall_curve, auc, roc_auc_score, average_precision_score
    
    fig, axes = plt.subplots(1, 2, figsize=(18, 7))
    
    # ROC Curve
    fpr, tpr, _ = roc_curve(y_test_best, y_prob_best)
    roc_auc = roc_auc_score(y_test_best, y_prob_best)
    
    axes[0].plot(fpr, tpr, color='#e74c3c', lw=3,
                label=f'ROC curve (AUC = {roc_auc:.4f})')
    axes[0].plot([0, 1], [0, 1], color='#95a5a6', lw=2, linestyle='--', label='Random Classifier')
    axes[0].set_xlim([0.0, 1.0])
    axes[0].set_ylim([0.0, 1.05])
    axes[0].set_xlabel('False Positive Rate', fontsize=13, fontweight='bold')
    axes[0].set_ylabel('True Positive Rate', fontsize=13, fontweight='bold')
    axes[0].set_title(f'ROC Curve - {best_model}', fontsize=14, fontweight='bold')
    axes[0].legend(loc="lower right", fontsize=12)
    axes[0].grid(True, alpha=0.3)
    
    # Precision-Recall Curve
    precision_curve, recall_curve, _ = precision_recall_curve(y_test_best, y_prob_best)
    pr_auc = average_precision_score(y_test_best, y_prob_best)
    
    axes[1].plot(recall_curve, precision_curve, color='#3498db', lw=3,
                label=f'PR curve (AUC = {pr_auc:.4f})')
    axes[1].set_xlabel('Recall', fontsize=13, fontweight='bold')
    axes[1].set_ylabel('Precision', fontsize=13, fontweight='bold')
    axes[1].set_title(f'Precision-Recall Curve - {best_model}', fontsize=14, fontweight='bold')
    axes[1].legend(loc="lower left", fontsize=12)
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\n📊 Curve Analysis:")
    print(f"   ROC-AUC: {roc_auc:.4f} - Overall discrimination ability")
    print(f"   PR-AUC:  {pr_auc:.4f} - Performance on imbalanced data")
else:
    print("\n⚠️  Probability scores not available for this model")

In [ ]:
# Error analysis - examine misclassified samples
print("\n" + "="*100)
print("🔍 ERROR ANALYSIS")
print("="*100)

# Get indices of misclassified samples
misclassified_idx = np.where(y_test_best != y_pred_best)[0]
false_positives_idx = np.where((y_test_best == 0) & (y_pred_best == 1))[0]
false_negatives_idx = np.where((y_test_best == 1) & (y_pred_best == 0))[0]

print(f"\nTotal misclassified samples: {len(misclassified_idx)} ({len(misclassified_idx)/len(y_test_best)*100:.2f}%)")
print(f"   False Positives: {len(false_positives_idx)} - Normal samples flagged as anomalies")
print(f"   False Negatives: {len(false_negatives_idx)} - Anomalies that were missed")

# Cost analysis (assuming costs)
cost_fp = 1.0  # Cost of false alarm
cost_fn = 10.0  # Cost of missed anomaly (usually higher)

total_cost = (len(false_positives_idx) * cost_fp + len(false_negatives_idx) * cost_fn)
print(f"\n💰 Cost Analysis (FP cost=${cost_fp}, FN cost=${cost_fn}):")
print(f"   Total cost: ${total_cost:.2f}")
print(f"   FP cost: ${len(false_positives_idx) * cost_fp:.2f}")
print(f"   FN cost: ${len(false_negatives_idx) * cost_fn:.2f}")

---
## 7. Feature Analysis <a id='features'></a>

Analyze feature importance and contribution to anomaly detection.

In [ ]:
# Feature importance analysis
print("\n" + "="*100)
print("🔍 FEATURE IMPORTANCE ANALYSIS")
print("="*100)

# Combine statistical significance with model insights
top_sig_features = sig_df.nsmallest(20, 'mannwhitney_p')

print("\nTop 20 Features by Statistical Significance:")
print(top_sig_features[['feature', 'cohens_d', 'effect_size', 'mannwhitney_p']].to_string(index=False))

In [ ]:
# Correlation matrix of top features
top_10_features = sig_df.nsmallest(10, 'mannwhitney_p')['feature'].values
corr_matrix = X_df[top_10_features].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
           center=0, square=True, linewidths=1,
           cbar_kws={'label': 'Correlation Coefficient'})
plt.title('Correlation Matrix - Top 10 Significant Features', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n💡 Note: High correlations (|r| > 0.8) may indicate redundant features")

In [ ]:
# Visualize feature distributions for top discriminative features
top_5_features = sig_df.nsmallest(5, 'mannwhitney_p')['feature'].values

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()

for idx, feature in enumerate(top_5_features):
    normal_data = X_df[y == 0][feature]
    anomaly_data = X_df[y == 1][feature]
    
    # Violin plot
    parts = axes[idx].violinplot([normal_data, anomaly_data], 
                                 positions=[0, 1],
                                 showmeans=True, showmedians=True)
    axes[idx].set_xticks([0, 1])
    axes[idx].set_xticklabels(['Normal', 'Anomaly'])
    axes[idx].set_ylabel('Value', fontsize=11)
    axes[idx].set_title(f'{feature}\n(Cohen\'s d = {sig_df[sig_df["feature"]==feature]["cohens_d"].values[0]:.3f})', 
                       fontsize=12, fontweight='bold')
    axes[idx].grid(alpha=0.3)

# Hide last subplot if odd number of features
if len(top_5_features) < 6:
    axes[5].axis('off')

plt.suptitle('Top 5 Discriminative Features - Distribution by Class', 
            fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

---
## 8. Production Deployment <a id='production'></a>

Prepare the model for production deployment.

In [ ]:
# Save the trained model
import joblib
from datetime import datetime

# Create model metadata
model_metadata = {
    'model_name': best_model,
    'training_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'n_training_samples': len(X_train),
    'n_test_samples': len(X_test),
    'features': feature_names,
    'scaling_method': 'robust',
    'imbalance_method': 'smote',
    'performance': {
        'accuracy': float(best_row['accuracy']),
        'precision': float(best_row['precision']),
        'recall': float(best_row['recall']),
        'f1_score': float(best_row['f1']),
        'roc_auc': float(best_row.get('roc_auc', 0))
    },
    'random_state': RANDOM_STATE
}

# Save model and metadata
joblib.dump(detector, 'anomaly_detector_model.pkl')
joblib.dump(model_metadata, 'model_metadata.pkl')

print("\n" + "="*100)
print("💾 MODEL SAVED FOR PRODUCTION")
print("="*100)
print("\nFiles saved:")
print("   1. anomaly_detector_model.pkl - Complete detector object")
print("   2. model_metadata.pkl - Model metadata and performance")

print("\n✅ Model ready for deployment!")

In [ ]:
# Demonstrate making predictions on new data
print("\n" + "="*100)
print("🔮 MAKING PREDICTIONS ON NEW DATA")
print("="*100)

# Generate some "new" data
X_new, y_new_true = make_classification(
    n_samples=100,
    n_features=N_FEATURES,
    n_informative=18,
    n_redundant=4,
    n_classes=2,
    weights=[0.88, 0.12],
    random_state=999
)

X_new_df = pd.DataFrame(X_new, columns=feature_names)

print(f"\nNew data shape: {X_new_df.shape}")

# Preprocess new data using the same transformer
X_new_proc, _ = detector.preprocess_data(X_new_df, X_new_df)

# Get predictions from all models
new_predictions, new_probabilities = detector.predict_all_models(X_new_proc)

# Use best model's predictions
final_predictions = new_predictions[best_model]

print(f"\n✅ Predictions generated using {best_model}")
print(f"   Predicted normal: {np.sum(final_predictions == 0)}")
print(f"   Predicted anomalies: {np.sum(final_predictions == 1)}")

# Create results DataFrame
results_df = X_new_df.copy()
results_df['prediction'] = final_predictions
results_df['prediction_label'] = ['Anomaly' if p == 1 else 'Normal' for p in final_predictions]

if best_model in new_probabilities:
    results_df['anomaly_score'] = new_probabilities[best_model]

print("\n📋 Sample Predictions (first 10):")
display_cols = ['prediction_label'] + feature_names[:3]
if 'anomaly_score' in results_df.columns:
    display_cols.append('anomaly_score')
    
results_df[display_cols].head(10)

In [ ]:
# Production inference example
print("\n" + "="*100)
print("🚀 PRODUCTION INFERENCE EXAMPLE")
print("="*100)

production_code = '''
# Production Inference Code
# ========================

import joblib
import pandas as pd

# Load the saved model
detector = joblib.load('anomaly_detector_model.pkl')
metadata = joblib.load('model_metadata.pkl')

# Load new data
X_new = pd.read_csv('new_data.csv')

# Ensure features match training
assert list(X_new.columns) == metadata['features'], "Feature mismatch!"

# Preprocess
X_new_proc, _ = detector.preprocess_data(X_new, X_new)

# Predict
predictions, probabilities = detector.predict_all_models(X_new_proc)
final_pred = predictions[metadata['model_name']]

# Get anomaly scores (if available)
if metadata['model_name'] in probabilities:
    anomaly_scores = probabilities[metadata['model_name']]
else:
    anomaly_scores = None

# Create output
output_df = X_new.copy()
output_df['is_anomaly'] = final_pred
output_df['anomaly_label'] = ['Anomaly' if p == 1 else 'Normal' for p in final_pred]
if anomaly_scores is not None:
    output_df['anomaly_score'] = anomaly_scores

# Save results
output_df.to_csv('predictions.csv', index=False)

print(f"Predictions complete! Found {sum(final_pred)} anomalies out of {len(final_pred)} samples.")
'''

print(production_code)

print("\n💡 Deployment Checklist:")
print("   ✅ Model saved and can be loaded")
print("   ✅ Metadata stored for validation")
print("   ✅ Preprocessing pipeline included")
print("   ✅ Feature names and order preserved")
print("   ✅ Performance metrics documented")

---
## 9. Summary & Next Steps <a id='summary'></a>

In [ ]:
# Create comprehensive summary
print("\n" + "="*100)
print("📊 PROJECT SUMMARY")
print("="*100)

print("\n🎯 Dataset:")
print(f"   Total samples: {len(X_df):,}")
print(f"   Features: {len(feature_names)}")
print(f"   Training samples: {len(X_train):,}")
print(f"   Test samples: {len(X_test):,}")
print(f"   Imbalance ratio: {np.sum(y == 0)/np.sum(y == 1):.2f}:1")

print("\n🔬 Statistical Analysis:")
print(f"   Significant features (p<0.05): {sig_df['is_significant'].sum()}/{len(sig_df)}")
print(f"   Features with large effect size: {sum(sig_df['effect_size'] == 'large')}")
print(f"   Non-normal features: {(~dist_df['is_normal_shapiro']).sum()}/{len(dist_df)}")

print("\n🤖 Models Evaluated:")
for model in comparison_df['model'].values:
    row = comparison_df[comparison_df['model'] == model].iloc[0]
    marker = "🏆" if model == best_model else "  "
    print(f"   {marker} {model}: F1={row['f1']:.4f}, Precision={row['precision']:.4f}, Recall={row['recall']:.4f}")

print(f"\n🏆 Best Model: {best_model.upper()}")
print(f"   Performance Metrics:")
print(f"      Accuracy: {best_row['accuracy']:.4f}")
print(f"      Precision: {best_row['precision']:.4f}")
print(f"      Recall: {best_row['recall']:.4f}")
print(f"      F1-Score: {best_row['f1']:.4f}")
print(f"      Balanced Accuracy: {best_row['balanced_accuracy']:.4f}")
if 'roc_auc' in best_row and best_row['roc_auc'] is not None:
    print(f"      ROC-AUC: {best_row['roc_auc']:.4f}")

print("\n💾 Deliverables:")
print("   ✅ Trained anomaly detection model")
print("   ✅ Model metadata and configuration")
print("   ✅ Statistical analysis results")
print("   ✅ Performance visualizations")
print("   ✅ Production-ready inference code")

print("\n📈 Key Findings:")
print(f"   1. Successfully handled {ratio:.1f}:1 class imbalance using SMOTE")
print(f"   2. {sig_df['is_significant'].sum()} features show statistical significance")
print(f"   3. {best_model} achieved best performance with F1={best_row['f1']:.4f}")
print(f"   4. Model correctly identifies {best_row['recall']*100:.1f}% of anomalies")
print(f"   5. False alarm rate: {(1-best_row.get('specificity', 0))*100:.1f}%")

print("\n🚀 Next Steps:")
print("   1. ✅ Deploy model to production environment")
print("   2. 📊 Monitor model performance on real data")
print("   3. 🔄 Retrain periodically with new data")
print("   4. 🎯 Fine-tune threshold based on business costs")
print("   5. 📈 Track false positive/negative rates")
print("   6. 🔍 Investigate misclassified samples for insights")

print("\n" + "="*100)
print("✅ ANALYSIS COMPLETE!")
print("="*100)

In [ ]:
# Final recommendations
print("\n💡 RECOMMENDATIONS FOR YOUR USE CASE:")
print("="*100)

print("\n1. Model Selection:")
if best_model == 'gaussian':
    print("   ✓ Gaussian model performed best - your data is well-suited for parametric methods")
    print("   → Consider PowerTransformer if features are skewed")
elif best_model == 'isolation_forest':
    print("   ✓ Isolation Forest performed best - good for general-purpose anomaly detection")
    print("   → Fast training and prediction, scales well")
elif best_model == 'one_class_svm':
    print("   ✓ One-Class SVM performed best - strong decision boundary")
    print("   → Good for clear separation between normal and anomalous")

print("\n2. Class Imbalance:")
if ratio > 10:
    print(f"   ⚠️  Severe imbalance ({ratio:.1f}:1)")
    print("   → Consider ADASYN or ensemble methods")
elif ratio > 5:
    print(f"   ⚠️  Moderate imbalance ({ratio:.1f}:1)")
    print("   → SMOTE is working well, continue using it")
else:
    print(f"   ✓ Manageable imbalance ({ratio:.1f}:1)")
    print("   → Current approach is appropriate")

print("\n3. Feature Engineering:")
n_significant = sig_df['is_significant'].sum()
if n_significant < len(feature_names) * 0.5:
    print(f"   ⚠️  Only {n_significant}/{len(feature_names)} features are significant")
    print("   → Consider feature selection to remove non-significant features")
else:
    print(f"   ✓ Good feature quality ({n_significant}/{len(feature_names)} significant)")

print("\n4. Performance Optimization:")
if best_row['recall'] < 0.8:
    print("   ⚠️  Low recall - missing some anomalies")
    print("   → Adjust decision threshold to increase sensitivity")
    print("   → Consider cost of missing anomalies vs false alarms")
if best_row['precision'] < 0.8:
    print("   ⚠️  Low precision - many false alarms")
    print("   → Adjust decision threshold to reduce false positives")
    print("   → Consider SMOTE-Tomek for better boundary separation")

print("\n5. Deployment:")
print("   ✓ Model is production-ready")
print("   ✓ Use provided inference code for deployment")
print("   → Set up monitoring for model drift")
print("   → Establish retraining schedule (e.g., monthly)")
print("   → Track performance metrics over time")

print("\n" + "="*100)

---

## 🎉 Congratulations!

You have successfully completed the **Robust Anomaly Detection System** workflow!

### What You've Accomplished:

✅ **Statistical Analysis**: Comprehensive hypothesis testing and effect size analysis  
✅ **Multiple Algorithms**: Trained and compared 5 different anomaly detection models  
✅ **Imbalance Handling**: Applied SMOTE to balance the dataset  
✅ **Model Evaluation**: Used 15+ metrics for thorough evaluation  
✅ **Feature Analysis**: Identified most important and significant features  
✅ **Production Ready**: Saved model and created deployment code  

### Files Generated:

- `anomaly_detector_model.pkl` - Trained model ready for deployment
- `model_metadata.pkl` - Model configuration and performance metrics
- Various visualization plots in the notebook

### Ready to Use With Your Data:

Simply replace the data generation code with your own data loading code:

```python
# Load your data
X_df = pd.read_csv('your_data.csv')
y = pd.read_csv('your_labels.csv').values.ravel()

# Run the pipeline
detector, results = run_robust_anomaly_detection(X_df, y)
```

---

**For questions or support, refer to:**
- README.md - Comprehensive documentation
- QUICK_REFERENCE.md - Common patterns and solutions
- PROJECT_OVERVIEW.md - Detailed project information

**Happy Anomaly Detecting! 🚀**